# L2, L1正則化（CIFAR10を用いた物体認識）

---
## 目的
畳み込みニューラルネットワーク (Convolutional Neural Network; CNN) を用いてCIFAR10データセットに対する物体認識を行う．
その際，L2正則化，L1正則化を用いることで，認識性能がどのように変化するかを確認する．

## 対応するチャプター
* 7.1.1: L2パラメータ正則化
* 7.1.2: L1正則化
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法
* 9.1: 畳み込み処理
* 9.3: プーリング


## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．

In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

## GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## データセットの読み込み
CIFAR10データセットを読み込みます．

読み込んだ学習データのサイズを確認します．
学習データは5万枚，1つのデータサイズは3x32x32の画像のような形式となっています．
これは32x32ピクセルのカラー画像という意味になります．

In [ ]:
train_data = torchvision.datasets.CIFAR10(root="./", train=True, transform=transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root="./", train=False, transform=transforms.ToTensor(), download=True)

print(train_data)
print(test_data)

## ネットワークモデルの定義
畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層2層，全結合層3層から構成されるネットワークとします．

1層目の畳み込み層は入力チャンネル数が1，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．
2層目の畳み込み層は入力チャネル数が16，出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．
1つ目の全結合層は入力ユニット数は特徴マップのサイズから，`8*8*32`とし，出力は1024としています．
次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．
また，ネットワーク内で使用する活性化関数を`self.act`に，プーリングを`self.pool`に設定します．
これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．
`forward`関数の引数`x`は入力データです．
それを`__init__`関数で定義した`conv1`に与え，その出力を活性化関数である`act`関数に与えます．
そして，その出力を`pool`に入力して，プーリング処理結果を`h`として出力します．
`h`は`conv2`に与えられて畳み込み処理とプーリング処理を行います．
そして，出力`h`を`l1`に与えて全結合層の処理を行います．
最終的に`l3`の全結合層の処理を行った出力`h`を戻り値としています．

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.l1 = nn.Linear(8 * 8 * 32, 1024)
        self.l2 = nn.Linear(1024, 1024)
        self.l3 = nn.Linear(1024, 10)
        self.act = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        h = self.pool(self.act(self.conv1(x)))
        h = self.pool(self.act(self.conv2(h)))
        h = h.view(h.size()[0], -1)
        h = self.act(self.l1(h))
        h = self.act(self.l2(h))
        h = self.l3(h)
        return h

## L1 正則化


## 学習
１回の誤差を算出するデータ数（ミニバッチサイズ）を64，学習エポック数を10とします．

また，L1正則化の度合いを制御するためのパラメータ`lam`を設定します．

学習を開始します．

学習を始める前に，`model.train()`を実行することで，ネットワークの演算を学習モードへ変更します．
学習モードへ変更することで，ネットワーク内の演算のうち，学習とテストで挙動が変化する演算を一括で変更することが可能です．
今回のネットワークではDropoutが該当し，学習モードの場合はDropoutが適用され，評価モード（詳細は後ほど紹介します）では，Dropoutによる結合の削除を行わずに推論を行うことが可能です．

各更新において，学習用データと教師データをそれぞれ`image`と`label`とします．
学習モデルにimageを与えて各クラスの確率yを取得します．
各クラスの確率yと教師ラベルtとの誤差を`criterion`で算出します．
また，分類結果に対する誤差に加えて，ネットワークパラメータのL1ノルムを計算し，損失関数に加えます．
このようにすることで，学習時の重みの更新にL1正則化を行うことが可能となります．
そして，誤差をbackward関数で逆伝播し， step関数でネットワークの更新を行います．

In [ ]:
# ネットワークモデル・最適化手法の設定
model_l1 = CNN()
if use_cuda:
    model_l1.cuda()

optimizer = torch.optim.SGD(model_l1.parameters(), lr=0.01, momentum=0.9)

# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# 正則化のスケールパラメータ
lam = 1e-4

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model_l1.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l1(image)
        
        loss = criterion(y, label)
        
        # パラメータのL1ノルムを損失関数に足す
        l1_loss = torch.tensor(0., requires_grad=True)
        for w in model_l1.parameters():
            l1_loss = l1_loss + torch.norm(w, 1)
        loss = loss + lam * l1_loss
        
        model_l1.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

## テスト
学習したネットワークモデルを用いて評価を行います．

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_l1.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l1(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## L2正則化

### 学習

次にL2正則化を用いた学習を行います．

まず，上のL1正則化と同様に，ネットワークモデルを初期化し，最適化手法，誤差関数などを設定します．

L2正則化では、通常の誤差に加えて，各パラメータのL2ノルムを足し合わせることで，正則化を行うことができます．

In [ ]:
# ネットワークモデル・最適化手法の設定
model_l2 = CNN()
if use_cuda:
    model_l2.cuda()

optimizer = torch.optim.SGD(model_l2.parameters(), lr=0.01, momentum=0.9)

# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# 正則化のスケールパラメータ
lam = 1e-4

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# ネットワークを学習モードへ変更
model_l2.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l2(image)
        
        loss = criterion(y, label)
        
        # パラメータのL2ノルムの二乗を損失関数に足す
        l2_loss = torch.tensor(0., requires_grad=True)
        for w in model_l2.parameters():
            l2_loss = l2_loss + torch.norm(w)**2
        loss = loss + lam * l2_loss
        
        model_l2.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

### テスト

In [ ]:
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_l2.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l2(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

### その他のL2正則化の方法

PyTorchでは，optimizerの設定時に，`weight_decay=***`を設定することで，L2正則化を用いた学習を行うことが可能となります．

In [ ]:
# ネットワークモデル
model_l2_wd = CNN()
if use_cuda:
    model_l2_wd.cuda()

# 最適化手法の設定（weight_decayでL2正則化を導入）
optimizer = torch.optim.SGD(model_l2_wd.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)

# ミニバッチサイズ・エポック数の設定
batch_size = 64
epoch_num = 10
n_iter = len(train_data) / batch_size

# データローダーの設定
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 誤差関数の設定
criterion = nn.CrossEntropyLoss()
if use_cuda:
    criterion.cuda()

# 学習 =========================================
# ネットワークを学習モードへ変更
model_l2_wd.train()

start = time()
for epoch in range(1, epoch_num+1):
    sum_loss = 0.0
    count = 0
    
    for image, label in train_loader:
        
        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l2_wd(image)
        
        loss = criterion(y, label)
        
        model_l2_wd.zero_grad()
        loss.backward()
        optimizer.step()
        
        sum_loss += loss.item()
        
        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)
        
    print("epoch: {}, mean loss: {}, mean accuracy: {}, elapsed_time :{}".format(epoch,
                                                                                 sum_loss / n_iter,
                                                                                 count.item() / len(train_data),
                                                                                 time() - start))

# テスト =======================================
# データローダーの準備
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

# ネットワークを評価モードへ変更
model_l2_wd.eval()

# 評価の実行
count = 0
with torch.no_grad():
    for image, label in test_loader:

        if use_cuda:
            image = image.cuda()
            label = label.cuda()
            
        y = model_l2_wd(image)

        pred = torch.argmax(y, dim=1)
        count += torch.sum(pred == label)

print("test accuracy: {}".format(count.item() / 10000.))

## 課題
1. 正則化の違いや有無で認識性能の変化を確認しましょう